In [ ]:
import jupyter
import ipywidgets as widgets
import numpy as np
import pandas as pd
from scipy import stats
from sklearn import model_selection
from sklearn.preprocessing import PowerTransformer
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# MODEL CREATION

# DATA HANDLING
# Get dataset
url = "https://raw.githubusercontent.com/JPeej/BodyFatCSV/main/bodyfat.csv"
df = pd.read_csv(url, sep=",")

# Drop unnecessary columns
df.drop(labels="Density", axis=1, inplace=True)

# Delete any rows with null values, no values are found but good practice if CSV is updated
df.dropna()

# Delete any duplicate entries, no values are found but good practice if CSV is updated
df.drop_duplicates()

# Drop entries with an outlier value
z_score = np.abs(stats.zscore(df))
df = df[(z_score < 3).all(axis=1)]

# Remove measurements that have weak correlation with bodyfat
df.drop(columns=['Height', 'Ankle', 'Wrist', 'Age'], axis=1, inplace=True)

# DEVELOP  MODEL
regressor = RandomForestRegressor(n_estimators=100, random_state=0)

# Get dependent(y, bodyfat) and independent(x, measurements) variables from split data
bodyfatIndex = df.columns.get_loc("BodyFat")
x_measurements = df.values[:, bodyfatIndex + 1:]
y_bodyfat = df.values[:, bodyfatIndex]
x_train, x_test, y_train, y_test = model_selection.train_test_split(x_measurements, y_bodyfat, test_size=0.2)

# Apply Yeo-Johnson Transformer
pt = PowerTransformer()
x_train = pt.fit_transform(x_train)
x_test = pt.transform(x_test)

# Train the model
regressor.fit(x_train, y_train);

In [ ]:
# USER INPUTS

weight = widgets.BoundedFloatText(
    min=0.0,
    max=400.0,
    disabled=False,
    continuous_update=True,
)
neck = widgets.BoundedFloatText(
    min=0.0,
    max=400.0,
    disabled=False,
    continuous_update=True
)
chest = widgets.BoundedFloatText(
    min=0.0,
    max=400.0,
    disabled=False,
    continuous_update=True
)
abdomen = widgets.BoundedFloatText(
    min=0.0,
    max=400.0,
    disabled=False,
    continuous_update=True
)
hip = widgets.BoundedFloatText(
    min=0.0,
    max=400.0,
    disabled=False,
    continuous_update=True
)
thigh = widgets.BoundedFloatText(
    min=0.0,
    max=400.0,
    disabled=False,
    continuous_update=True
)
knee = widgets.BoundedFloatText(
    min=0.0,
    max=400.0,
    disabled=False,
    continuous_update=True
)
bicep = widgets.BoundedFloatText(
    min=0.0,
    max=400.0,
    disabled=False,
    continuous_update=True
)
forearm = widgets.BoundedFloatText(
    min=0.0,
    max=400.0,
    disabled=False,
    continuous_update=True
)

In [ ]:
# SUBMIT BUTTON

submit_output = widgets.Label("Estimated bodyfat = xx.xx")

def on_submit_clicked(button):
    metrics = [[weight.value, neck.value, chest.value, abdomen.value, hip.value, thigh.value, knee.value, bicep.value, forearm.value]]
    bodyfat = regressor.predict(metrics)
    submit_output.value = f"Estimated bodyfat = {bodyfat}"

submit_button = widgets.Button(description='Click for results')
submit_button.on_click(on_submit_clicked)

In [ ]:
widgets.AppLayout\
    (header=print('\033[1m' + "Body Fat Estimator\nThis app provides an estimation of the user's body fat percentage based upon their body metric inputs.\nInput the metrics required below in numeric values only.\nOnce complete, click the submit button. Upon submit, the algorithm will determine the user's estimated body fat percentage and display it.\nIf the user wishes to try again please refresh the page." + '\033[1m'),
    left_sidebar=widgets.VBox(
        [widgets.Label('Weight in lb : ', layout=widgets.Layout(height='36px')),
         widgets.Label('Neck circumference in cm : ', layout=widgets.Layout(height='36px')),
         widgets.Label('Chest circumference in cm : ',layout=widgets.Layout(height='36px')),
         widgets.Label('Abdomen circumference in cm : ', layout=widgets.Layout(height='36px')),
         widgets.Label('Hip circumference in cm : ', layout=widgets.Layout(height='36px')),
         widgets.Label('Thigh circumference in cm : ', layout=widgets.Layout(height='36px')),
         widgets.Label('Knee circumference in cm : ', layout=widgets.Layout(height='36px')),
         widgets.Label('Bicep circumference in cm : ', layout=widgets.Layout(height='36px')),
         widgets.Label('Forearm circumference in cm : ', layout=widgets.Layout(height='36px'))]),
      center=widgets.VBox([weight, neck, chest, abdomen, hip, thigh, knee, bicep, forearm]),
      right_sidebar=widgets.VBox([submit_button, submit_output]),
      footer=None)